# Se carga el SET de audios

In [5]:
import os
import glob
import numpy as np
import pandas as pd

# Obtener todas las rutas de archivos dentro de train/ con sus subdirectorios
#audios = glob.glob("./Raw/*/*.wav", recursive=True)

# /teamspace/studios/opensoundscape/extended_audios_chiffchaff/*.wav
# /teamspace/studios/opensoundscape/extended_audios_littleowl/*.wav
# /teamspace/studios/opensoundscape/extended_audios_pipit/*.wav
audios_chiffchaff = glob.glob("/teamspace/studios/opensoundscape/extended_audios_chiffchaff/*.wav", recursive=True)
audios_littleowl = glob.glob("/teamspace/studios/opensoundscape/extended_audios_littleowl/*.wav", recursive=True)
audios_pipit = glob.glob("/teamspace/studios/opensoundscape/extended_audios_pipit/*.wav", recursive=True)
audios_rtbc = glob.glob("/teamspace/studios/opensoundscape/extended_audios_rtbc/*.wav", recursive=True)
audios_littlepenguin = glob.glob("/teamspace/studios/opensoundscape/extended_audios_littlepenguin/*/*.wav", recursive=True)
# Verificar cuántos archivos se encontraron correctamente
print(f"Archivos encontrados: Audios chiffchaff: {len(audios_chiffchaff)}, Audios littleowl: {len(audios_littleowl)}, Audios pipit: {len(audios_pipit)}, Audios rtbc: {len(audios_rtbc)}, Audios LP: {len(audios_littlepenguin)},")
print(audios_chiffchaff[:5])  # Verificar las primeras rutas generadas
print(audios_littleowl[:5])  # Verificar las primeras rutas generadas
print(audios_pipit[:5])  # Verificar las primeras rutas generadas
print(audios_rtbc[:5])  # Verificar las primeras rutas generadas
print(audios_littlepenguin[:5])  # Verificar las primeras rutas generadas

# Nombre del archivo donde se guardarán los embeddings
csv_chiffchaff = "chiffchaff_embeddings.csv"
csv_littleowl = "littleowl_embeddings.csv"
csv_pipit = "pipit_embeddings.csv"
csv_rtbc = "rtbc_embeddings.csv"
csv_littlepenguin = "littlepenguin_embeddings.csv"


audios = audios_littlepenguin
csv_filename = csv_littlepenguin

Archivos encontrados: Audios chiffchaff: 6762, Audios littleowl: 952, Audios pipit: 1364, Audios rtbc: 1785, Audios LP: 2429,
['/teamspace/studios/opensoundscape/extended_audios_chiffchaff/cutted_day1_PC1101_0000.wav', '/teamspace/studios/opensoundscape/extended_audios_chiffchaff/cutted_day1_PC1101_0001.wav', '/teamspace/studios/opensoundscape/extended_audios_chiffchaff/cutted_day1_PC1101_0002.wav', '/teamspace/studios/opensoundscape/extended_audios_chiffchaff/cutted_day1_PC1101_0003.wav', '/teamspace/studios/opensoundscape/extended_audios_chiffchaff/cutted_day1_PC1101_0004.wav']
['/teamspace/studios/opensoundscape/extended_audios_littleowl/littleowl2017fg_test_108_0000.wav', '/teamspace/studios/opensoundscape/extended_audios_littleowl/littleowl2017fg_test_108_0001.wav', '/teamspace/studios/opensoundscape/extended_audios_littleowl/littleowl2017fg_test_108_0002.wav', '/teamspace/studios/opensoundscape/extended_audios_littleowl/littleowl2017fg_test_108_0003.wav', '/teamspace/studios/open

# Se utiliza birdnet lib para extraer Embeddings de los audios. Continúa donde iba

¿Qué cambia en este código?
- ✅ Verifica si el archivo CSV ya existe. Si existe, carga los archivos procesados y los almacena en processed_files.
- ✅ Si el código se detiene, al reiniciarlo solo procesará los audios faltantes. Antes de procesar un audio, revisa si ya está en processed_files.
- ✅ Guarda los datos en el CSV después de cada audio. Usa mode='a' (append) para agregar nuevas filas sin sobrescribir el archivo.
- ✅ Evita problemas de memoria

🔹 Nuevas mejoras en el código
- ✅ Cuenta cuántos audios se han procesado en esta ejecución. La variable processed_count se incrementa después de cada audio.
- ✅ Cada 500 audios muestra un mensaje en pantalla- if processed_count % 500 == 0: imprime el número de audios procesados hasta ese momento.
- ✅ Sigue siendo seguro para interrupciones


Después de guardar en CSV, limpia la lista all_data.
Con este enfoque, si el código se detiene en cualquier momento, puedes simplemente volver a ejecutarlo y continuará desde el último audio procesado. 🚀

El problema es que es algo lento... porque solo usa un hilo de CPU

In [6]:
from birdnetlib import Recording
from birdnetlib.analyzer import Analyzer
import pandas as pd
import os

# Nombre del archivo donde se guardarán los embeddings
# csv_filename = "chiffchaff_embeddings.csv"

# Inicializar el modelo BirdNET
analyzer = Analyzer()

# Verificar si ya existe el archivo CSV para recuperar los audios procesados
if os.path.exists(csv_filename):
    df_existing = pd.read_csv(csv_filename)
    processed_files = set(df_existing["file_name"])  # Archivos ya procesados
else:
    df_existing = pd.DataFrame()
    processed_files = set()

# Lista de rutas de los archivos de audio
# Asegúrate de que 'audios' ya está definida y contiene las rutas correctas

all_data = []
processed_count = 0  # Contador de archivos procesados en esta ejecución

# Recorrer cada archivo en la lista de audios
for audio_path in audios:
    try:
        # Extraer el nombre del archivo sin la ruta
        file_name = os.path.basename(audio_path)

        # Saltar si el archivo ya fue procesado
        if file_name in processed_files:
            print(f"Saltando {file_name}, ya procesado.")
            continue

        # Cargar la grabación en BirdNET
        recording = Recording(analyzer, audio_path)
        recording.extract_embeddings()

        # Procesar los embeddings extraídos
        for emb in recording.embeddings:
            row = {
                "path": audio_path,  # Ruta completa del archivo
                "file_name": file_name,  # Nombre del archivo sin la ruta
                "start_time": emb['start_time'],
                "end_time": emb['end_time']
            }
            # Añadir los 1024 valores del embedding
            for i, value in enumerate(emb["embeddings"]):
                row[f"dim_{i+1}"] = value

            all_data.append(row)

        # Guardar en el archivo CSV después de procesar cada audio
        df_partial = pd.DataFrame(all_data)
        df_partial.to_csv(csv_filename, mode='a', header=not os.path.exists(csv_filename), index=False)
        
        # Limpiar la lista para ahorrar memoria
        all_data = []

        # Incrementar contador
        processed_count += 1

        # Mostrar mensaje cada 500 audios procesados
        if processed_count % 500 == 0:
            print(f"{processed_count} audios procesados hasta ahora.")

    except Exception as e:
        print(f"Error procesando {audio_path}: {e}")

print(f"Embeddings guardados en {csv_filename}")


Labels loaded.
load model True
Model loaded.
Labels loaded.
load_species_list_model
Meta model loaded.
read_audio_data
read_audio_data: complete, read  1 chunks.
extract_embeddings_for_recording 13B3-1_exhale-i10_10.wav
read_audio_data
read_audio_data: complete, read  1 chunks.
extract_embeddings_for_recording 13B3-1_exhale-i10_77.wav
read_audio_data
read_audio_data: complete, read  1 chunks.
extract_embeddings_for_recording 13B3-1_exhale-i11_11.wav
read_audio_data
read_audio_data: complete, read  1 chunks.
extract_embeddings_for_recording 13B3-1_exhale-i11_78.wav
read_audio_data
read_audio_data: complete, read  1 chunks.
extract_embeddings_for_recording 13B3-1_exhale-i12_12.wav
read_audio_data
read_audio_data: complete, read  1 chunks.
extract_embeddings_for_recording 13B3-1_exhale-i12_79.wav
read_audio_data
read_audio_data: complete, read  1 chunks.
extract_embeddings_for_recording 13B3-1_exhale-i13_13.wav
read_audio_data
read_audio_data: complete, read  1 chunks.
extract_embeddings_

# Contar el número de audios procesados para ver si todos quedaron con embeddings.

In [7]:
import pandas as pd
import os

# Nombre del archivo donde se guardarán los embeddings
# csv_filename = "chiffchaff_embeddings.csv"

# Verificar si ya existe el archivo CSV para recuperar los audios procesados
if os.path.exists(csv_filename):
    df_embeddings = pd.read_csv(csv_filename)
    processed_files = set(df_embeddings["file_name"])  # Archivos ya procesados
else:
    df_embeddings = pd.DataFrame()
    processed_files = set()

num_audios_procesados = df_embeddings["path"].nunique()  # O el nombre correcto de la columna
print(f"Se lograron procesar {num_audios_procesados} audios diferentes.")

# Convertir la lista original de audios a un conjunto
audios_originales = set(audios)

# Obtener la lista de audios procesados del DataFrame
audios_procesados = set(df_embeddings["path"].unique())

# Identificar los audios que no se procesaron
audios_no_procesados = audios_originales - audios_procesados

# Mostrar resultados
print(f"Total de audios en la lista original: {len(audios_originales)}")
print(f"Total de audios procesados: {len(audios_procesados)}")
print(f"Total de audios no procesados: {len(audios_no_procesados)}")

# Si quieres ver qué audios no se procesaron:
print("Audios no procesados:")
print("\n".join(audios_no_procesados))


Se lograron procesar 2429 audios diferentes.
Total de audios en la lista original: 2429
Total de audios procesados: 2429
Total de audios no procesados: 0
Audios no procesados:



In [8]:
print (df_embeddings.shape)

(2630, 1028)
